In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.utils import plot_model


In [2]:
# from numba import cuda 
# device = cuda.get_current_device()
# device.reset()

In [3]:
img = np.load('./data/images.npy')
lbl = np.load('./data/labels.npy')
print(f"{img.shape=}  {lbl.shape=}")

img.shape=(18000, 150, 150)  lbl.shape=(18000, 2)


In [4]:
img = img.astype(float) / 255
img = np.expand_dims(img, -1)

print(f"{img.shape=}  {lbl.shape=}")

img.shape=(18000, 150, 150, 1)  lbl.shape=(18000, 2)


In [5]:
hours = lbl[:,0]
num_classes = 12
hours = keras.utils.to_categorical(hours, num_classes)
minutes = lbl[:,1]/60
print(minutes[25])
input_shape = (150, 150, 1)

0.016666666666666666


In [6]:
x_train, x_test, hours_train, hours_test, minutes_train, minutes_test = train_test_split(img, hours, minutes, train_size=0.8, random_state=42)

In [7]:
def create_convblock(filters, n, layer):
    for _ in range(n):
        layer = keras.layers.Conv2D(filters, kernel_size=(3, 3), activation='relu',padding='same')(layer)
    return layer

def create_model():
    input_layer = keras.layers.Input(shape=(input_shape))
    layer = create_convblock(32,2,input_layer)
    max_layer = keras.layers.MaxPooling2D(pool_size=(2, 2))(layer)
    layer = create_convblock(64,2,max_layer)
    max_layer = keras.layers.MaxPooling2D(pool_size=(2, 2))(layer)
    layer = create_convblock(128,3,max_layer)
    max_layer = keras.layers.MaxPooling2D(pool_size=(2, 2))(layer)
    layer = create_convblock(256,3,max_layer)
    split_layer = keras.layers.MaxPooling2D(pool_size=(2, 2))(layer)
    
    minutes_output = keras.layers.Flatten()(split_layer)
    minutes_output = keras.layers.Dropout(0.1)(minutes_output)
    minutes_output =  keras.layers.Dense(1, activation='sigmoid',name="minutes_output")(minutes_output)

    hours_output = keras.layers.Flatten()(split_layer)
    hours_output =  keras.layers.Dropout(0.1)(hours_output)
    hours_output =  keras.layers.Dense(num_classes, activation='softmax', name="hours_output")(hours_output)
    return  keras.Model(inputs=[input_layer],outputs=[hours_output,minutes_output])

model = create_model()

In [8]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 150, 150, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 150, 150, 32  320         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 150, 150, 32  9248        ['conv2d[0][0]']                 
                                )                                                             

In [9]:
# # compile models
model.compile(loss= {'hours_output': 'categorical_crossentropy', 'minutes_output': 'mse'},
            optimizer=keras.optimizers.Adam(learning_rate=0.1),
            metrics= {'hours_output': 'accuracy', 'minutes_output': 'mse'}
            )

In [10]:
# y_train = y_train.astype(float)
# y_test = y_test.astype(float) 

In [11]:
with tf.device('/gpu:0'):
    model.fit(x_train, [hours_train, minutes_train], epochs=10,
            batch_size = 128,
            
        )

Epoch 1/10
112/112 [==============================] - 48s 317ms/step - loss: 56268345344.0000 - hours_output_loss: 56268345344.0000 - minutes_output_loss: 0.1329 - hours_output_accuracy: 0.0824 - minutes_output_mse: 0.1329 - val_loss: 2.5506 - val_hours_output_loss: 2.4814 - val_minutes_output_loss: 0.0692 - val_hours_output_accuracy: 0.0972 - val_minutes_output_mse: 0.0692
Epoch 2/10
  1/112 [..............................] - ETA: 32s - loss: 2.5627 - hours_output_loss: 2.4840 - minutes_output_loss: 0.0787 - hours_output_accuracy: 0.0469 - minutes_output_mse: 0.0787

KeyboardInterrupt: 

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])

NameError: name 'y_test' is not defined

In [ ]:
print(score)

79.16700744628906


In [ ]:
x_test[0].shape

(150, 150, 1)

In [ ]:
y_train[89]

312.0

In [ ]:
model.predict(X_train[89].reshape(1, 150, 150, 1))

1/1 [==============================] - 0s 16ms/step


array([[354.89505]], dtype=float32)